In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import StratifiedKFold,KFold


# 导入需要的模块

import matplotlib.pyplot as plt

# 用来绘图的，封装了matplot
# 要注意的是一旦导入了seaborn，
# matplotlib的默认作图风格就会被覆盖成seaborn的格式
import seaborn as sns       

from scipy import stats
from scipy.stats import  norm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline  
# 为了在jupyter notebook里作图，需要用到这个命令

In [2]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 4.8 MB/s eta 0:00:00


In [3]:
data_path = r'../input/trainA/trainA.xlsx'
train_data = pd.read_excel(data_path,sheet_name='训练集')
test_data = pd.read_excel(data_path,sheet_name='测试集')
train_data.head()

,报价编号,距离km,车型,起始城市,起始城市代码,目的城市,目的城市代码,是否跨省运输,车长,车宽,...,运输要求,是否有要求,备注,是否有备注,车价,油价,耗油价,柴油价,柴油耗油价,translation
0,XJ2022041400009,559.18,微型面包车,陕西省,610,陕西省,610.0,0.0,1.8,1.3,...,NaN,0,货物共计18件，160公斤，库房内装车，按照客户要求货物卸到指定位置，返回单,38,77.78,147.0,82199.460,8.510,4758.62180,"The goods a total of 18, 160 kg, warehouse loa..."
1,2021102000011,67.17,7.6米厢车,重庆市,500,重庆市,500.0,0.0,7.6,2.3,...,NaN,0,NaN,0,81.48,159.7,10727.049,7.620,511.83540,#VALUE!
2,2021111100008,548.07,4.2米厢车,济南市,370,田家庵区,340.0,1.0,4.2,1.8,...,NaN,0,项目要求使用9.6和4.2货车各1辆，约600公里。,26,74.07,168.5,92349.795,7.530,4126.96710,1 each project requires the use of 9.6 and 4.2...
3,XJ2022042000004,263.95,16.5米厢车,福州市,350,漳州市,350.0,0.0,16.5,2.5,...,NaN,0,NaN,0,129.63,147.0,38800.650,8.510,2246.21450,#VALUE!
4,2022012500017,175.97,高顶依维柯,上海市,310,浙江省,330.0,1.0,2.3,1.6,...,NaN,0,费用含装卸，一装两卸，单件货物比较大最好两个人送货，还要有手拉车，因为要上楼,38,62.52,138.2,24319.054,7.755,1364.64735,"Fees including loading and unloading, the unlo..."


In [4]:
# 填充第一个众数
train_data['车宽']= train_data['车宽'].fillna(train_data['车宽'].mode()[0])
train_data['车高']= train_data['车高'].fillna(train_data['车高'].mode()[0])
train_data['是否跨省运输']= train_data['是否跨省运输'].fillna(train_data['是否跨省运输'].mode()[0])
train_data['车辆容积']= train_data['车辆容积'].fillna(train_data['车辆容积'].mode()[0])
train_data['标准载重']= train_data['标准载重'].fillna(train_data['标准载重'].mode()[0])
train_data['货物类型']= train_data['货物类型'].fillna(train_data['货物类型'].mode()[0])

In [5]:
test_data['车宽']= test_data['车宽'].fillna(test_data['车宽'].mode()[0])
test_data['车高']= test_data['车高'].fillna(test_data['车高'].mode()[0])
test_data['是否跨省运输']= test_data['是否跨省运输'].fillna(test_data['是否跨省运输'].mode()[0])
test_data['货物类型']= test_data['货物类型'].fillna(test_data['货物类型'].mode()[0])
test_data['车辆容积']= test_data['车辆容积'].fillna(test_data['车辆容积'].mode()[0])
test_data['标准载重']= test_data['标准载重'].fillna(test_data['标准载重'].mode()[0])

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2629 entries, 0 to 2628
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   报价编号         2629 non-null   object 
 1   距离km         2629 non-null   float64
 2   车型           2629 non-null   object 
 3   起始城市         2629 non-null   object 
 4   起始城市代码       2586 non-null   object 
 5   目的城市         2629 non-null   object 
 6   目的城市代码       2583 non-null   float64
 7   是否跨省运输       2629 non-null   float64
 8   车长           2629 non-null   float64
 9   车宽           2629 non-null   float64
 10  车高           2629 non-null   float64
 11  标准载重         2629 non-null   float64
 12  车辆容积         2629 non-null   float64
 13  货物类型         2629 non-null   object 
 14  用车时间         2629 non-null   object 
 15  货物特性         64 non-null     object 
 16  运输要求         520 non-null    object 
 17  是否有要求        2629 non-null   int64  
 18  备注           1812 non-null   object 
 19  是否有备注 

In [7]:
# # 时间处理(提取年月日)
train_data['用车时间'] = pd.to_datetime(train_data['用车时间'])
test_data['用车时间'] = pd.to_datetime(test_data['用车时间'])


train_data['year'] = train_data['用车时间'].dt.year
train_data['month'] = train_data['用车时间'].dt.month
train_data['day'] = train_data['用车时间'].dt.day


test_data['year'] = test_data['用车时间'].dt.year
test_data['month'] = test_data['用车时间'].dt.month
test_data['day'] = test_data['用车时间'].dt.day


In [8]:
# 此部分先使用了log1p，还没对比其他的数据转换方法
train_data['距离km'] = np.log1p(train_data['距离km'])
test_data['距离km'] = np.log1p(test_data['距离km'])
train_data['车价'] = np.log1p(train_data['车价'])
test_data['车价'] = np.log1p(test_data['车价'])
train_data['车长'] = np.log1p(train_data['车长'])
test_data['车长'] = np.log1p(test_data['车长'])

train_data['车宽'] = np.log1p(train_data['车宽'])
test_data['车宽'] = np.log1p(test_data['车宽'])

train_data['车高'] = np.log1p(train_data['车高'])
test_data['车高'] = np.log1p(test_data['车高'])

train_data['车宽'] = np.log1p(train_data['车宽'])
test_data['车宽'] = np.log1p(test_data['车宽'])

train_data['车辆容积'] = np.log1p(train_data['车辆容积'])
test_data['车辆容积'] = np.log1p(test_data['车辆容积'])

train_data['是否有要求'] = np.log1p(train_data['是否有要求'])
test_data['是否有要求'] = np.log1p(test_data['是否有要求'])

train_data['是否有备注'] = np.log1p(train_data['是否有备注'])
test_data['是否有备注'] = np.log1p(test_data['是否有备注'])

train_data['耗油价'] = np.log1p(train_data['耗油价'])
test_data['耗油价'] = np.log1p(test_data['耗油价'])

train_data['柴油耗油价'] = np.log1p(train_data['柴油耗油价'])
test_data['柴油耗油价'] = np.log1p(test_data['柴油耗油价'])

train_data['标准载重'] = np.log1p(train_data['标准载重'])
test_data['标准载重'] = np.log1p(test_data['标准载重'])

train_data['柴油价'] = np.log1p(train_data['柴油价'])
test_data['柴油价'] = np.log1p(test_data['柴油价'])

train_data['油价'] = np.log1p(train_data['油价'])
test_data['油价'] = np.log1p(test_data['油价'])

# train_data['month'] = np.log1p(train_data['month'])
# test_data['month'] = np.log1p(test_data['month'])

# train_data['day'] = np.log1p(train_data['day'])
# test_data['day'] = np.log1p(test_data['day'])

# train_data['year'] = np.log1p(train_data['year'])
# test_data['year'] = np.log1p(test_data['year'])

In [9]:
data_all = pd.concat([train_data, test_data],ignore_index=True, sort = False)

In [10]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
# data_all['便利设施'] = data_all['便利设施'].apply(
#     lambda x: x.replace('{', '').replace('}', '').replace('"', '').replace(':', '').replace(',', ' '))
# # df_features['便利设施'] = df_features['便利设施'].str.lower()
def jiebasplit(x):
    generator = jieba.cut(str(x))
    
    return ' '.join(generator)

data_all['备注'] = data_all['备注'].apply(jiebasplit)
# df_features['便利设施'] = df_features['便利设施'].str.lower()
n_components = 12

X = list(data_all['备注'].values)
tfv = TfidfVectorizer(ngram_range=(1,1), max_features=10000)
tfv.fit(X)
X_tfidf = tfv.transform(X)
svd = TruncatedSVD(n_components= n_components)
svd.fit(X_tfidf)
X_svd = svd.transform(X_tfidf)

for i in range(n_components):
    data_all[f'备注_tfidf_{i}'] = X_svd[:, i]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.908 seconds.
Prefix dict has been built successfully.


In [11]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
# data_all['便利设施'] = data_all['便利设施'].apply(
#     lambda x: x.replace('{', '').replace('}', '').replace('"', '').replace(':', '').replace(',', ' '))
# # df_features['便利设施'] = df_features['便利设施'].str.lower()
def jiebasplit(x):
    generator = jieba.cut(str(x))
    
    return ' '.join(generator)

data_all['运输要求'] = data_all['运输要求'].apply(jiebasplit)
# df_features['便利设施'] = df_features['便利设施'].str.lower()
n_components = 12

X = list(data_all['运输要求'].values)
tfv = TfidfVectorizer(ngram_range=(1,1), max_features=10000)
tfv.fit(X)
X_tfidf = tfv.transform(X)
svd = TruncatedSVD(n_components= n_components)
svd.fit(X_tfidf)
X_svd = svd.transform(X_tfidf)

for i in range(n_components):
    data_all[f'运输要求_tfidf_{i}'] = X_svd[:, i]

In [12]:
classfied_pro = {'东部':{'北京':110,'天津':120,'河北':130,'上海':310,"江苏":320,'浙江':330,'福建':350,'山东':370,'广东':440,'海南':460,'台湾':710,'香港':810,'澳门':820},#东部地区
                '中部':{'山西':140,'安徽':340,'江西':360,'河南':410,'湖北':420,'湖南':430},# 中部
                '西部':{'内蒙古':150,'广西':450,'重庆':500,'四川':510,'贵州':520,'云南':530,'西藏':540,'陕西':610,'甘肃':620,'青海':630,'宁夏':640,'新疆':650},#西部
                '东北':{'辽宁':210,'吉林':220,'黑龙江':230}}#东北
print(len(classfied_pro))
def extract_pro(pro_code):
    for j in classfied_pro:
        for key,value in classfied_pro[j].items():
            if  pro_code == value:
                return j
                break
data_all['到达区域'] = data_all['目的城市代码'].apply(extract_pro)
data_all['起始区域'] = data_all['起始城市代码'].apply(extract_pro)
data_all.head()

4


,报价编号,距离km,车型,起始城市,起始城市代码,目的城市,目的城市代码,是否跨省运输,车长,车宽,...,运输要求_tfidf_4,运输要求_tfidf_5,运输要求_tfidf_6,运输要求_tfidf_7,运输要求_tfidf_8,运输要求_tfidf_9,运输要求_tfidf_10,运输要求_tfidf_11,到达区域,起始区域
0,XJ2022041400009,6.328258,微型面包车,陕西省,610,陕西省,610.0,0.0,1.029619,0.605904,...,1.007355e-18,-2.924640e-18,1.189936e-18,-5.681944e-19,-9.144480e-19,-1.368522e-16,-6.570483e-17,4.615438e-18,西部,西部
1,2021102000011,4.222005,7.6米厢车,重庆市,500,重庆市,500.0,0.0,2.151762,0.785691,...,1.007355e-18,-2.924640e-18,1.189936e-18,-5.681944e-19,-9.144480e-19,-1.368522e-16,-6.570483e-17,4.615438e-18,西部,西部
2,2021111100008,6.308226,4.2米厢车,济南市,370,田家庵区,340.0,1.0,1.648659,0.707848,...,1.007355e-18,-2.924640e-18,1.189936e-18,-5.681944e-19,-9.144480e-19,-1.368522e-16,-6.570483e-17,4.615438e-18,中部,东部
3,XJ2022042000004,5.579541,16.5米厢车,福州市,350,漳州市,350.0,0.0,2.862201,0.812157,...,1.007355e-18,-2.924640e-18,1.189936e-18,-5.681944e-19,-9.144480e-19,-1.368522e-16,-6.570483e-17,4.615438e-18,东部,东部
4,2022012500017,5.175980,高顶依维柯,上海市,310,浙江省,330.0,1.0,1.193922,0.670652,...,1.007355e-18,-2.924640e-18,1.189936e-18,-5.681944e-19,-9.144480e-19,-1.368522e-16,-6.570483e-17,4.615438e-18,东部,东部


In [13]:
df = data_all.copy()

In [14]:

oil_data = pd.read_excel(data_path,sheet_name='油价')


def find_time(objtime,oiltype):
    t = oil_data['调整日期']
    right = len(t)
    left = 0
    mid = (left+right)//2
    while left<right-1:
        if t[mid]<objtime:
            right = mid-1
            
        else:
            left = mid+1
        mid = (right+left)//2
    a = t[right]
    res = oil_data[oil_data['调整日期'].isin([a])]
    if oiltype == '汽油':
        o = '价格(元/吨)'
    else:
        o = '价格(元/吨).1'
    res =  res[o]
    if '万' in str(res):
        res = res.values
        res = res[0][:-1]
        res = float(res)*10000
    else:
        res = res.values[0]
    return res

In [15]:
df['汽油'] = df['用车时间'].apply(find_time,args=('汽油',))
df['柴油'] = df['用车时间'].apply(find_time,args=('柴油',))

In [16]:

oil_data = pd.read_excel(data_path,sheet_name='油价')

def find_time(objtime,oiltype):
    t = oil_data['调整日期']
    right = len(t)
    left = 0
    mid = (left+right)//2
    while left<right-1:
        if t[mid]<objtime:
            right = mid-1
            
        else:
            left = mid+1
        mid = (right+left)//2
    a = t[right]
    res = oil_data[oil_data['调整日期'].isin([a])]
    if oiltype == '汽油':
        o = '涨跌'
    else:
        o = '涨跌.1'
    res =  res[o]
    if '↑' in str(res):
        res = res.values
        res = res[0][1:]
        res = float(res)
    else:
        res = res.values
        res = res[0][1:]
        res = -1*float(res)
    
    return res
df['汽油_涨跌'] = df['用车时间'].apply(find_time,args=('汽油',))
df['柴油_涨跌'] = df['用车时间'].apply(find_time,args=('柴油',))

In [17]:
# scenes = df["运输要求"].unique().tolist()
# scenes
# requires = []
# for x in scenes[1:]:  
#     x = x.split(',')
#     for i in x:
#         if i in requires:
#             continue
#         else:
#             requires.append(i)
# print(requires)
# len(requires)

# def extract_req(x,name):

#     if type(x) == float:
#         return  0
#     else:    
#         x.split(',')
#         if name in x:
#             return 1
#         else :
#             return 0
# for i in requires:
#     df[i] = df['运输要求'].apply(extract_req,args=(i,))
# df.head()

In [18]:
for feat in ['车型','货物类型','货物特性'
#              ,'运输要求'
             ,'到达区域','起始区域'
            ]:# 返回列标签
#     if df[feat].dtype == object:
    df[feat] = df[feat].astype('str')
    print(feat)
    le = LabelEncoder()
    df[feat] = le.fit_transform(df[feat])

车型
货物类型
货物特性
到达区域
起始区域


In [19]:
df.columns

Index(['报价编号', '距离km', '车型', '起始城市', '起始城市代码', '目的城市', '目的城市代码', '是否跨省运输',
       '车长', '车宽', '车高', '标准载重', '车辆容积', '货物类型', '用车时间', '货物特性', '运输要求',
       '是否有要求', '备注', '是否有备注', '车价', '油价', '耗油价', '柴油价', '柴油耗油价',
       'translation', 'year', 'month', 'day', '备注_tfidf_0', '备注_tfidf_1',
       '备注_tfidf_2', '备注_tfidf_3', '备注_tfidf_4', '备注_tfidf_5', '备注_tfidf_6',
       '备注_tfidf_7', '备注_tfidf_8', '备注_tfidf_9', '备注_tfidf_10', '备注_tfidf_11',
       '运输要求_tfidf_0', '运输要求_tfidf_1', '运输要求_tfidf_2', '运输要求_tfidf_3',
       '运输要求_tfidf_4', '运输要求_tfidf_5', '运输要求_tfidf_6', '运输要求_tfidf_7',
       '运输要求_tfidf_8', '运输要求_tfidf_9', '运输要求_tfidf_10', '运输要求_tfidf_11',
       '到达区域', '起始区域', '汽油', '柴油', '汽油_涨跌', '柴油_涨跌'],
      dtype='object')

In [20]:
for feat in df.columns:
    if df[feat].dtype == object:
        df[feat] = df[feat].astype('str')
        print(feat)
        le = LabelEncoder()
        df[feat] = le.fit_transform(df[feat])

报价编号
起始城市
起始城市代码
目的城市
运输要求
备注
translation


In [21]:
df.columns

Index(['报价编号', '距离km', '车型', '起始城市', '起始城市代码', '目的城市', '目的城市代码', '是否跨省运输',
       '车长', '车宽', '车高', '标准载重', '车辆容积', '货物类型', '用车时间', '货物特性', '运输要求',
       '是否有要求', '备注', '是否有备注', '车价', '油价', '耗油价', '柴油价', '柴油耗油价',
       'translation', 'year', 'month', 'day', '备注_tfidf_0', '备注_tfidf_1',
       '备注_tfidf_2', '备注_tfidf_3', '备注_tfidf_4', '备注_tfidf_5', '备注_tfidf_6',
       '备注_tfidf_7', '备注_tfidf_8', '备注_tfidf_9', '备注_tfidf_10', '备注_tfidf_11',
       '运输要求_tfidf_0', '运输要求_tfidf_1', '运输要求_tfidf_2', '运输要求_tfidf_3',
       '运输要求_tfidf_4', '运输要求_tfidf_5', '运输要求_tfidf_6', '运输要求_tfidf_7',
       '运输要求_tfidf_8', '运输要求_tfidf_9', '运输要求_tfidf_10', '运输要求_tfidf_11',
       '到达区域', '起始区域', '汽油', '柴油', '汽油_涨跌', '柴油_涨跌'],
      dtype='object')

In [22]:
def extract(x):
    return float(x)

col1 = [ '距离km',]
col2 =[ '车长', '车宽','车高','标准载重', '车辆容积','货物类型','起始区域','到达区域','标准载重', '车辆容积','month','year','day','汽油','柴油','汽油_涨跌','柴油_涨跌',]
for i in col1:
    for j in col2:
        if i != j:
            df['{}*{}'.format(i,j)] = df[i].apply(extract)*df[j].apply(extract)

In [23]:
df.columns

Index(['报价编号', '距离km', '车型', '起始城市', '起始城市代码', '目的城市', '目的城市代码', '是否跨省运输',
       '车长', '车宽', '车高', '标准载重', '车辆容积', '货物类型', '用车时间', '货物特性', '运输要求',
       '是否有要求', '备注', '是否有备注', '车价', '油价', '耗油价', '柴油价', '柴油耗油价',
       'translation', 'year', 'month', 'day', '备注_tfidf_0', '备注_tfidf_1',
       '备注_tfidf_2', '备注_tfidf_3', '备注_tfidf_4', '备注_tfidf_5', '备注_tfidf_6',
       '备注_tfidf_7', '备注_tfidf_8', '备注_tfidf_9', '备注_tfidf_10', '备注_tfidf_11',
       '运输要求_tfidf_0', '运输要求_tfidf_1', '运输要求_tfidf_2', '运输要求_tfidf_3',
       '运输要求_tfidf_4', '运输要求_tfidf_5', '运输要求_tfidf_6', '运输要求_tfidf_7',
       '运输要求_tfidf_8', '运输要求_tfidf_9', '运输要求_tfidf_10', '运输要求_tfidf_11',
       '到达区域', '起始区域', '汽油', '柴油', '汽油_涨跌', '柴油_涨跌', '距离km*车长', '距离km*车宽',
       '距离km*车高', '距离km*标准载重', '距离km*车辆容积', '距离km*货物类型', '距离km*起始区域',
       '距离km*到达区域', '距离km*month', '距离km*year', '距离km*day', '距离km*汽油',
       '距离km*柴油', '距离km*汽油_涨跌', '距离km*柴油_涨跌'],
      dtype='object')

## 统计特征

In [24]:
df['货物类型'].value_counts()

0    2055
2     460
1     414
Name: 货物类型, dtype: int64

In [25]:
df.columns

Index(['报价编号', '距离km', '车型', '起始城市', '起始城市代码', '目的城市', '目的城市代码', '是否跨省运输',
       '车长', '车宽', '车高', '标准载重', '车辆容积', '货物类型', '用车时间', '货物特性', '运输要求',
       '是否有要求', '备注', '是否有备注', '车价', '油价', '耗油价', '柴油价', '柴油耗油价',
       'translation', 'year', 'month', 'day', '备注_tfidf_0', '备注_tfidf_1',
       '备注_tfidf_2', '备注_tfidf_3', '备注_tfidf_4', '备注_tfidf_5', '备注_tfidf_6',
       '备注_tfidf_7', '备注_tfidf_8', '备注_tfidf_9', '备注_tfidf_10', '备注_tfidf_11',
       '运输要求_tfidf_0', '运输要求_tfidf_1', '运输要求_tfidf_2', '运输要求_tfidf_3',
       '运输要求_tfidf_4', '运输要求_tfidf_5', '运输要求_tfidf_6', '运输要求_tfidf_7',
       '运输要求_tfidf_8', '运输要求_tfidf_9', '运输要求_tfidf_10', '运输要求_tfidf_11',
       '到达区域', '起始区域', '汽油', '柴油', '汽油_涨跌', '柴油_涨跌', '距离km*车长', '距离km*车宽',
       '距离km*车高', '距离km*标准载重', '距离km*车辆容积', '距离km*货物类型', '距离km*起始区域',
       '距离km*到达区域', '距离km*month', '距离km*year', '距离km*day', '距离km*汽油',
       '距离km*柴油', '距离km*汽油_涨跌', '距离km*柴油_涨跌'],
      dtype='object')

In [26]:
number_col = ['距离km', '油价', '耗油价', '柴油价', '柴油耗油价', '汽油', '柴油',] # '货物类型', 
group =  '车型'
data_sum = df.groupby(group)[number_col].sum().reset_index()
data_sum.columns = [group]+[group+'_'+i+'_sum' for i in number_col]
data_mean = df.groupby(group)[number_col].mean().reset_index()
data_mean.columns = [group]+[group+'_'+i+'_mean' for i in number_col]
data_std = df.groupby(group)[number_col].std().reset_index()
data_std.columns = [group]+[group+'_'+i+'_std' for i in number_col]

data_max = df.groupby(group)[number_col].max().reset_index()
data_max.columns = [group]+[group+'_'+i+'_max' for i in number_col]

data_min = df.groupby(group)[number_col].min().reset_index()
data_min.columns = [group]+[group+'_'+i+'_min' for i in number_col]
df = df.merge(data_sum,how='left',on = group)
df = df.merge(data_mean,how='left',on = group)
df = df.merge(data_std,how='left',on = group)
df = df.merge(data_max,how='left',on = group)
df = df.merge(data_min,how='left',on = group)

In [27]:
df

,报价编号,距离km,车型,起始城市,起始城市代码,目的城市,目的城市代码,是否跨省运输,车长,车宽,...,车型_柴油耗油价_max,车型_汽油_max,车型_柴油_max,车型_距离km_min,车型_油价_min,车型_耗油价_min,车型_柴油价_min,车型_柴油耗油价_min,车型_汽油_min,车型_柴油_min
0,2702,6.328258,28,146,33,543,610.0,0.0,1.029619,0.605904,...,9.324230,10115.0,9040,1.532557,4.935912,6.288305,2.050913,3.276488,7910.0,6915
1,1128,4.222005,22,138,28,511,500.0,0.0,2.151762,0.785691,...,9.901387,10115.0,9040,1.800058,4.935912,6.575905,2.050913,3.717777,7910.0,6915
2,1405,6.308226,14,94,18,398,340.0,1.0,1.648659,0.707848,...,10.192611,10115.0,9040,1.175573,4.935912,5.760516,2.050913,2.783529,7910.0,6915
3,2779,5.579541,7,118,15,378,350.0,0.0,2.862201,0.812157,...,10.090776,10115.0,9040,2.678965,4.935912,7.575406,2.050913,4.564244,7910.0,6915
4,2146,5.175980,30,1,10,338,330.0,1.0,1.193922,0.670652,...,10.028285,10115.0,9040,0.000000,4.935912,0.000000,2.050913,0.000000,7910.0,6915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,2418,1.644805,15,66,29,87,510.0,0.0,1.648659,0.799206,...,9.334923,10115.0,9040,1.644805,4.935912,6.425083,2.078191,3.646165,7910.0,6915
2925,2739,2.862201,14,53,18,175,370.0,0.0,1.648659,0.707848,...,10.192611,10115.0,9040,1.175573,4.935912,5.760516,2.050913,2.783529,7910.0,6915
2926,399,4.083115,19,81,13,304,330.0,0.0,2.054124,0.785691,...,10.106432,10115.0,9040,2.913980,4.935912,7.813720,2.050913,4.890356,7910.0,6915
2927,840,7.511732,14,94,18,399,440.0,1.0,1.648659,0.707848,...,10.192611,10115.0,9040,1.175573,4.935912,5.760516,2.050913,2.783529,7910.0,6915


In [28]:
number_col = ['距离km', '油价', '耗油价', '柴油价', '柴油耗油价', '汽油', '柴油',] # '货物类型', 
group =  '货物类型'
data_sum = df.groupby(group)[number_col].sum().reset_index()
data_sum.columns = [group]+[group+'_'+i+'_sum' for i in number_col]
data_mean = df.groupby(group)[number_col].mean().reset_index()
data_mean.columns = [group]+[group+'_'+i+'_mean' for i in number_col]
data_std = df.groupby(group)[number_col].std().reset_index()
data_std.columns = [group]+[group+'_'+i+'_std' for i in number_col]

data_max = df.groupby(group)[number_col].max().reset_index()
data_max.columns = [group]+[group+'_'+i+'_max' for i in number_col]

data_min = df.groupby(group)[number_col].min().reset_index()
data_min.columns = [group]+[group+'_'+i+'_min' for i in number_col]
df = df.merge(data_sum,how='left',on = group)
df = df.merge(data_mean,how='left',on = group)
df = df.merge(data_std,how='left',on = group)
df = df.merge(data_max,how='left',on = group)
df = df.merge(data_min,how='left',on = group)

In [29]:
df.columns

Index(['报价编号', '距离km', '车型', '起始城市', '起始城市代码', '目的城市', '目的城市代码', '是否跨省运输',
       '车长', '车宽',
       ...
       '货物类型_柴油耗油价_max', '货物类型_汽油_max', '货物类型_柴油_max', '货物类型_距离km_min',
       '货物类型_油价_min', '货物类型_耗油价_min', '货物类型_柴油价_min', '货物类型_柴油耗油价_min',
       '货物类型_汽油_min', '货物类型_柴油_min'],
      dtype='object', length=144)

In [30]:
# #定义交叉特征统计
# from tqdm import tqdm
# def cross_cat_num(df, num_col, cat_col):
#     for f1 in tqdm(cat_col):
#         g = df.groupby(f1, as_index=False)
#         for f2 in tqdm(num_col):
#             if f1==f2:
#                 continue
#             else:
#                 feat = g[f2].agg({
#                     '{}_{}_max'.format(f1, f2): 'max', '{}_{}_min'.format(f1, f2): 'min',
#                     '{}_{}_median'.format(f1, f2): 'median',
#                     '{}_{}_sum'.format(f1, f2): 'sum',
#                     '{}_{}_mad'.format(f1, f2): 'mad',
#                 })
#                 df = df.merge(feat, on=f1, how='left')
#     return(df)

# #cross_cat = ['车型','货物类型','是否跨省运输', 'year']
# cross_cat = [ '距离km', '车长', '标准载重', '车辆容积','起始区域','到达区域']
# #cross_num = ['距离km', '车长', '车宽', '车高', '标准载重', '车辆容积']
# cross_num =['汽油','chaiyou','标准载重', '车辆容积']
# data2 = cross_cat_num(df, cross_num, cross_cat)  # 一阶交叉
# data2.shape

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2929 entries, 0 to 2928
Columns: 144 entries, 报价编号 to 货物类型_柴油_min
dtypes: datetime64[ns](1), float64(121), int64(22)
memory usage: 3.2 MB


In [32]:
train_data = df[:-300].copy()
test_data = df[-300:].copy()

# large_val_cols = ['汽油','柴油']
# for c in large_val_cols:
#     train_data[c] = np.log(train_data[c] + 1)
# for c in large_val_cols:
#     test_data[c] = np.log(test_data[c] + 1)
    
    
# trd = pd.read_excel(data_path,sheet_name='训练集')
# ted = pd.read_excel(data_path,sheet_name='测试集')

# train_data['translation'] = trd['translation']
# test_data['translation'] = ted['translation']
    
train_data.to_csv('./clear_train_jike_all_feature2.csv',index=0)
test_data.to_csv('./clear_test_jike_all_feature2.csv', index=0)

In [33]:
feats = [f for f in train_data.columns if f not in ['运输要求','day', '标准载重', '用车时间','货物特性', 'index','车价', "是否跨省运输",'报价编号',
                                                    '起始城市代码','目的城市代码''城市代码','城市代码','translation','备注']]

In [34]:
feats


['距离km',
 '车型',
 '起始城市',
 '目的城市',
 '目的城市代码',
 '车长',
 '车宽',
 '车高',
 '车辆容积',
 '货物类型',
 '是否有要求',
 '是否有备注',
 '油价',
 '耗油价',
 '柴油价',
 '柴油耗油价',
 'year',
 'month',
 '备注_tfidf_0',
 '备注_tfidf_1',
 '备注_tfidf_2',
 '备注_tfidf_3',
 '备注_tfidf_4',
 '备注_tfidf_5',
 '备注_tfidf_6',
 '备注_tfidf_7',
 '备注_tfidf_8',
 '备注_tfidf_9',
 '备注_tfidf_10',
 '备注_tfidf_11',
 '运输要求_tfidf_0',
 '运输要求_tfidf_1',
 '运输要求_tfidf_2',
 '运输要求_tfidf_3',
 '运输要求_tfidf_4',
 '运输要求_tfidf_5',
 '运输要求_tfidf_6',
 '运输要求_tfidf_7',
 '运输要求_tfidf_8',
 '运输要求_tfidf_9',
 '运输要求_tfidf_10',
 '运输要求_tfidf_11',
 '到达区域',
 '起始区域',
 '汽油',
 '柴油',
 '汽油_涨跌',
 '柴油_涨跌',
 '距离km*车长',
 '距离km*车宽',
 '距离km*车高',
 '距离km*标准载重',
 '距离km*车辆容积',
 '距离km*货物类型',
 '距离km*起始区域',
 '距离km*到达区域',
 '距离km*month',
 '距离km*year',
 '距离km*day',
 '距离km*汽油',
 '距离km*柴油',
 '距离km*汽油_涨跌',
 '距离km*柴油_涨跌',
 '车型_距离km_sum',
 '车型_油价_sum',
 '车型_耗油价_sum',
 '车型_柴油价_sum',
 '车型_柴油耗油价_sum',
 '车型_汽油_sum',
 '车型_柴油_sum',
 '车型_距离km_mean',
 '车型_油价_mean',
 '车型_耗油价_mean',
 '车型_柴油价_mean',
 '车型_柴油耗油价_mean',
 '车型_汽油_mean',


In [35]:
for feat in df.columns:
    if df[feat].dtype == object:
        df[feat] = df[feat].astype('str')
        print(feat)
        le = LabelEncoder()
        df[feat] = le.fit_transform(df[feat])
train_data = df[:-300].copy()
test_data = df[-300:].copy()


In [36]:
params = {
        'learning_rate':0.05,
        "objective": "regression",
        "metric": "rmse",
        'boosting_type': "gbdt",
        'verbosity': -1,
        'n_jobs': -1, 
        'seed': 2022,
        'lambda_l1': 6.610898817934583, 
        'lambda_l2': 1.2572931636397838e-07, 
        'num_leaves': 122, 
        'feature_fraction': 0.709347964783973, 
        'bagging_fraction': 0.9368697249173327, 
        'bagging_freq': 6, 
        'max_depth': 7, 
        'max_bin': 214, 
        'min_data_in_leaf': 450,
        'n_estimators': 1000, 
    }
    

In [37]:
train_data.columns

Index(['报价编号', '距离km', '车型', '起始城市', '起始城市代码', '目的城市', '目的城市代码', '是否跨省运输',
       '车长', '车宽',
       ...
       '货物类型_柴油耗油价_max', '货物类型_汽油_max', '货物类型_柴油_max', '货物类型_距离km_min',
       '货物类型_油价_min', '货物类型_耗油价_min', '货物类型_柴油价_min', '货物类型_柴油耗油价_min',
       '货物类型_汽油_min', '货物类型_柴油_min'],
      dtype='object', length=144)

In [38]:
train_data['货物类型']

0       0
1       2
2       2
3       1
4       1
       ..
2624    0
2625    0
2626    2
2627    0
2628    0
Name: 货物类型, Length: 2629, dtype: int64

In [39]:
train_data= pd.read_csv('./clear_train_jike_all_feature2.csv')
test_data= pd.read_csv('./clear_test_jike_all_feature2.csv')
scaler = StandardScaler()
feats = ['距离km','车型','车长', '车宽', '车高','是否跨省运输',
         '标准载重', '车辆容积','油价', '耗油价', '柴油价', '柴油耗油价',
        '备注_tfidf_0', '备注_tfidf_1',
       '备注_tfidf_2', '备注_tfidf_3', '备注_tfidf_4', '备注_tfidf_5', '备注_tfidf_6',
       '备注_tfidf_7', '备注_tfidf_8', '备注_tfidf_9', '备注_tfidf_10', '备注_tfidf_11',
        '距离km*车长', '距离km*车宽','距离km*车高', '距离km*标准载重', '距离km*车辆容积', '距离km*货物类型', '距离km*起始区域',
       '距离km*到达区域', '距离km*month', '距离km*year', '距离km*day', '距离km*汽油',
       '距离km*柴油', '距离km*汽油_涨跌', '距离km*柴油_涨跌']

train_data[feats] = scaler.fit_transform(train_data[feats])
test_data[feats] = scaler.fit_transform(test_data[feats])
train_data.head()

,报价编号,距离km,车型,起始城市,起始城市代码,目的城市,目的城市代码,是否跨省运输,车长,车宽,...,货物类型_柴油耗油价_max,货物类型_汽油_max,货物类型_柴油_max,货物类型_距离km_min,货物类型_油价_min,货物类型_耗油价_min,货物类型_柴油价_min,货物类型_柴油耗油价_min,货物类型_汽油_min,货物类型_柴油_min
0,2702,0.456903,1.423175,146,33,543,610.0,-1.420693,-1.723880,-2.086022,...,10.240574,10115.0,9040,0.000000,4.935912,0.000000,2.050913,0.000000,7910.0,6915
1,1128,-0.846174,0.670454,138,28,511,500.0,-1.420693,0.231832,0.601813,...,10.471416,10115.0,9040,0.000000,4.935912,0.000000,2.050913,0.000000,7910.0,6915
2,1405,0.444510,-0.333175,94,18,398,340.0,0.703882,-0.644996,-0.561947,...,10.471416,10115.0,9040,0.000000,4.935912,0.000000,2.050913,0.000000,7910.0,6915
3,2779,-0.006306,-1.211350,118,15,378,350.0,-1.420693,1.470010,0.997489,...,10.509044,10115.0,9040,1.289233,4.935912,5.950455,2.050913,2.964433,7910.0,6915
4,2146,-0.255977,1.674083,1,10,338,330.0,0.703882,-1.437526,-1.118040,...,10.509044,10115.0,9040,1.289233,4.935912,5.950455,2.050913,2.964433,7910.0,6915


In [40]:

scaler = StandardScaler()


train_data[feats] = scaler.fit_transform(train_data[feats])
test_data[feats] = scaler.fit_transform(test_data[feats])
train_data.head()

,报价编号,距离km,车型,起始城市,起始城市代码,目的城市,目的城市代码,是否跨省运输,车长,车宽,...,货物类型_柴油耗油价_max,货物类型_汽油_max,货物类型_柴油_max,货物类型_距离km_min,货物类型_油价_min,货物类型_耗油价_min,货物类型_柴油价_min,货物类型_柴油耗油价_min,货物类型_汽油_min,货物类型_柴油_min
0,2702,0.456903,1.423175,146,33,543,610.0,-1.420693,-1.723880,-2.086022,...,10.240574,10115.0,9040,0.000000,4.935912,0.000000,2.050913,0.000000,7910.0,6915
1,1128,-0.846174,0.670454,138,28,511,500.0,-1.420693,0.231832,0.601813,...,10.471416,10115.0,9040,0.000000,4.935912,0.000000,2.050913,0.000000,7910.0,6915
2,1405,0.444510,-0.333175,94,18,398,340.0,0.703882,-0.644996,-0.561947,...,10.471416,10115.0,9040,0.000000,4.935912,0.000000,2.050913,0.000000,7910.0,6915
3,2779,-0.006306,-1.211350,118,15,378,350.0,-1.420693,1.470010,0.997489,...,10.509044,10115.0,9040,1.289233,4.935912,5.950455,2.050913,2.964433,7910.0,6915
4,2146,-0.255977,1.674083,1,10,338,330.0,0.703882,-1.437526,-1.118040,...,10.509044,10115.0,9040,1.289233,4.935912,5.950455,2.050913,2.964433,7910.0,6915


In [41]:
fold_num = 5
seeds = [2222]
oof = np.zeros(len(train_data))
importance = 0
LABEL = '车价'
pred_y = pd.DataFrame()
for seed in seeds:
    kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_data[feats], train_data[LABEL])):
        print('-----------', fold)
        train = lgb.Dataset(train_data.loc[train_idx, feats],
                            train_data.loc[train_idx, LABEL])
        val = lgb.Dataset(train_data.loc[val_idx, feats],
                          train_data.loc[val_idx, LABEL])
        model = lgb.train(params, train, valid_sets=val, num_boost_round=10000,
                          early_stopping_rounds=100, verbose_eval=200)
        

        
        oof[val_idx] += model.predict(train_data.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(test_data[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
pd.set_option("display.max_rows",None)
test_df_list = list(test_data[LABEL])
predict = np.expm1(pred_y.mean(axis=1))
true = np.expm1(test_df_list)


def error_rate(true_label,pred_label):
    res = abs(pred_label-true_label)/true_label
#     print(res.shape())
    return res.mean()


result = error_rate(true,predict)
result

----------- 0
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.408841
[400]	valid_0's rmse: 0.400407
[600]	valid_0's rmse: 0.395246
[800]	valid_0's rmse: 0.391753
[1000]	valid_0's rmse: 0.389595
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.389595
----------- 1
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.415901
[400]	valid_0's rmse: 0.407916
[600]	valid_0's rmse: 0.403541
[800]	valid_0's rmse: 0.400192
[1000]	valid_0's rmse: 0.399035
Did not meet early stopping. Best iteration is:
[984]	valid_0's rmse: 0.398703
----------- 2
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.409664
[400]	valid_0's rmse: 0.403575
[600]	valid_0's rmse: 0.399163
[800]	valid_0's rmse: 0.395677
[1000]	valid_0's rmse: 0.393493
Did not meet early stopping. Best iteration is:
[990]	valid_0's rmse: 0.393421
----------- 3
Training until validation scores don't improve for 1

0.2447382755964259

In [42]:
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
print(feats_importance.sort_values('importance', ascending=False)[:50])

           name    importance
25      距离km*车宽  10787.651161
24      距离km*车长   4608.498517
26      距离km*车高   4485.751288
35      距离km*汽油   2083.537032
11        柴油耗油价    552.608225
27    距离km*标准载重    393.242301
28    距离km*车辆容积    284.769255
36      距离km*柴油    268.430585
2            车长    260.210390
6          标准载重    244.643992
30    距离km*起始区域    173.147783
3            车宽     98.039253
1            车型     94.326001
0          距离km     79.853443
12   备注_tfidf_0     66.183027
4            车高     65.112219
5        是否跨省运输     62.195443
33    距离km*year     54.691052
32   距离km*month     49.794841
31    距离km*到达区域     48.387341
8            油价     45.909066
15   备注_tfidf_3     26.061649
7          车辆容积     21.025200
13   备注_tfidf_1     19.713514
34     距离km*day     19.369397
14   备注_tfidf_2     17.847840
9           耗油价     17.345818
17   备注_tfidf_5     16.725542
37   距离km*汽油_涨跌     11.101392
38   距离km*柴油_涨跌      9.658116
23  备注_tfidf_11      9.048345
29    距离km*货物类型      7.453019
20   备注_tf

In [43]:
top30 = feats_importance.sort_values('importance', ascending=False)[:30]
top30
feature30 = top30['name'].values
feature30

array(['距离km*车宽', '距离km*车长', '距离km*车高', '距离km*汽油', '柴油耗油价', '距离km*标准载重',
       '距离km*车辆容积', '距离km*柴油', '车长', '标准载重', '距离km*起始区域', '车宽', '车型',
       '距离km', '备注_tfidf_0', '车高', '是否跨省运输', '距离km*year', '距离km*month',
       '距离km*到达区域', '油价', '备注_tfidf_3', '车辆容积', '备注_tfidf_1', '距离km*day',
       '备注_tfidf_2', '耗油价', '备注_tfidf_5', '距离km*汽油_涨跌', '距离km*柴油_涨跌'],
      dtype=object)

In [44]:
top20 = feats_importance.sort_values('importance', ascending=False)[:20]
feature20 = top20['name'].values
feature20

array(['距离km*车宽', '距离km*车长', '距离km*车高', '距离km*汽油', '柴油耗油价', '距离km*标准载重',
       '距离km*车辆容积', '距离km*柴油', '车长', '标准载重', '距离km*起始区域', '车宽', '车型',
       '距离km', '备注_tfidf_0', '车高', '是否跨省运输', '距离km*year', '距离km*month',
       '距离km*到达区域'], dtype=object)

In [45]:
def error_rate(true_label,pred_label):
    res = abs(pred_label-true_label)/true_label
#     print(res.shape())
    return res.mean()

In [46]:
pd.set_option("display.max_rows",None)
test_df_list = list(test_data[LABEL])
predict = np.expm1(pred_y.mean(axis=1))
true = np.expm1(test_df_list)
result = error_rate(true,predict)
result

0.2447382755964259

In [47]:
train_data = pd.read_csv('./clear_train_jike_all_feature.csv')
test_data = pd.read_csv('./clear_test_jike_all_feature.csv')
print(train_data.columns)
# del train_data['报价编号']
# del train_data['起始城市']
# # del df['起始城市代码']
# # del df['目的城市代码']
# del train_data['目的城市']
# del train_data['备注']
# # del df['运输要求']
# del train_data['translation']


# # del test_data['报价编号']
# # del test_data['起始城市']
# # # del df['起始城市代码']
# # # del df['目的城市代码']
# # del test_data['目的城市']
# # del test_data['备注']
# # # del df['运输要求']
# del test_data['translation']
choosed_fea = ['车价','距离km']
train_data = train_data[choosed_fea]
test_data = test_data[choosed_fea]
train_data.to_csv('./clear_train_jike_v3.csv',index=0)
test_data.to_csv('./clear_test_jike_v3.csv', index=0)


FileNotFoundError: [Errno 2] No such file or directory: './clear_train_jike_all_feature.csv'

In [ ]:
train_data = pd.read_csv('./clear_train_jike_all_feature.csv')
test_data = pd.read_csv('./clear_test_jike_all_feature.csv')
print(train_data.columns)
# del train_data['报价编号']
# del train_data['起始城市']
# # del df['起始城市代码']
# # del df['目的城市代码']
# del train_data['目的城市']
# del train_data['备注']
# # del df['运输要求']
# del train_data['translation']


# # del test_data['报价编号']
# # del test_data['起始城市']
# # # del df['起始城市代码']
# # # del df['目的城市代码']
# # del test_data['目的城市']
# # del test_data['备注']
# # # del df['运输要求']
# del test_data['translation']
choosed_fea = ['车价','距离km','起始城市', '起始城市代码', '目的城市', '目的城市代码','车型','车长', '车宽', '车高',]
train_data = train_data[choosed_fea]
test_data = test_data[choosed_fea]
train_data.to_csv('./clear_train_jike_v3.csv',index=0)
test_data.to_csv('./clear_test_jike_v3.csv', index=0)


In [ ]:
!pip install -U "mxnet<2.0.0"
!pip install autogluon

In [ ]:
t = pd.read_csv('./clear_train_jike_v3.csv')
t.columns

In [ ]:
train_data.columns

In [ ]:
# from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
from autogluon.tabular import TabularDataset, TabularPredictor

# hyperparameters = get_hyperparameter_config('multimodal')

from autogluon.tabular import TabularPredictor

train_data = TabularDataset('./clear_train_jike_v3.csv')
label = '车价'

predictor = TabularPredictor(label=label).fit(train_data
                                              ,hyperparameters = 'multimodal'
#                                               ,num_stack_levels = 1, num_bag_folds = 5
                                             )
test_data = TabularDataset('./clear_test_jike_v3.csv')
preds = predictor.predict(test_data)
preds.shape
test_df = pd.read_csv('./clear_test_jike_v3.csv')
test_df_list = list(test_df['车价'])
true = np.expm1(test_df_list)
predict = np.expm1(preds)
def error_rate(true_label,pred_label):
    res = abs(pred_label-true_label)/true_label
#     print(res.shape())
    return res.mean()

# result = error_rate(true,preds)
result = error_rate(true,predict)
result

In [ ]:
test_data = TabularDataset('./clear_test_jike_v3.csv')
preds = predictor.predict(test_data)
preds.shape
test_df = pd.read_csv('../input/cleardatav3/clear_test_jike_v3.csv')
test_df_list = list(test_df['车价'])
true = np.expm1(test_df_list)
predict = np.expm1(preds)
def error_rate(true_label,pred_label):
    res = abs(pred_label-true_label)/true_label
#     print(res.shape())
    return res.mean()

# result = error_rate(true,preds)
result = error_rate(true,predict)
result

In [ ]:
test_df = pd.read_csv('../input/cleardatav3/clear_test_jike_v3.csv')
test_df_list = list(test_df['车价'])
true = np.expm1(test_df_list)
predict = np.expm1(preds)
def error_rate(true_label,pred_label):
    res = abs(pred_label-true_label)/true_label
#     print(res.shape())
    return res.mean()

# result = error_rate(true,preds)
result = error_rate(true,predict)
result

In [ ]:
test_data.head()

In [ ]:
predict.to_csv('./result_v3.csv')

v1 0.1874  
v2 0.195            
v3  0.1915                   lgb 0.242

## 其他模型

In [ ]:
#train_data = pd.read_csv('./clear_train_jike_all_feature.csv')
y_train = train_data['车价']
train = train_data[['距离km*车宽', '距离km*车高', '距离km*车长', '距离km*汽油', '柴油耗油价', '距离km*标准载重','距离km*车辆容积', '距离km*柴油']]
train.columns

In [ ]:
y_train.values

In [ ]:
del train['运输要求']
del train['备注']
del train['translation']

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train.values, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [ ]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred)

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15